In [ ]:
!pip install -q langchain_google_genai

In [20]:
!pip install -q moviepy langchain  faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.0 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-community

In [12]:
import os
os.environ["GOOGLE_API_KEY"]=""
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0,

)

In [2]:
!pip install -q yt-dlp

# Example Python code to download video
import yt_dlp

url = "https://www.youtube.com/watch?v=hmqYxByTlRs"

ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.4 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=hmqYxByTlRs
[youtube] hmqYxByTlRs: Downloading webpage
[youtube] hmqYxByTlRs: Downloading tv simply player API JSON
[youtube] hmqYxByTlRs: Downloading tv client config
[youtube] hmqYxByTlRs: Downloading player 6742b2b9-main
[youtube] hmqYxByTlRs: Downloading tv player API JSON
[info] hmqYxByTlRs: Downloading 1 format(s): 247+251
[download] Destination: Create Simple RAG based AI Chat bot ｜ Python+LangChain+FAISS Vector Database(No API Keys Needed!).f247.webm
[download] 100% of    7.91MiB in 00:00:00 at 9.20MiB/s   
[download] Destination: Create Simple RAG based AI Chat bot ｜ Python+LangChain+FAISS Vector Database(No API Keys Needed!).f251.webm
[download] 100% of    6.56MiB in 00:00:01 at 6.02MiB/s   
[Merger] Merging formats into "Create Simple RAG based AI Chat bot ｜ Pyth

In [9]:
import yt_dlp

def download_youtube_video(url):
    """
    Download a YouTube video with user-selected resolution.
    Falls back to highest available if requested resolution is not present.
    """
    # Ask user for preferred resolution
    resolution = input("Enter desired resolution (e.g., 1080, 720, 480): ")

    # yt-dlp format string: video + audio best matching resolution
    ydl_opts = {
        'format': f'bestvideo[height<={resolution}]+bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'merge_output_format': 'mp4'
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Successfully downloaded!")
    except Exception as e:
        print("Error downloading video:", e)





In [33]:
url = "https://www.youtube.com/watch?v=wrpmqMz-m4w"
download_youtube_video(url)

Enter desired resolution (e.g., 1080, 720, 480): 480
[youtube] Extracting URL: https://www.youtube.com/watch?v=wrpmqMz-m4w
[youtube] wrpmqMz-m4w: Downloading webpage
[youtube] wrpmqMz-m4w: Downloading tv simply player API JSON
[youtube] wrpmqMz-m4w: Downloading tv client config
[youtube] wrpmqMz-m4w: Downloading tv player API JSON
[info] wrpmqMz-m4w: Downloading 1 format(s): 397+251
[download] Destination: This 2 Minute Challenge can make you a Topper⚡.f397.mp4
[download] 100% of    3.60MiB in 00:00:00 at 4.70MiB/s   
[download] Destination: This 2 Minute Challenge can make you a Topper⚡.f251.webm
[download] 100% of    1.98MiB in 00:00:00 at 25.11MiB/s  
[Merger] Merging formats into "This 2 Minute Challenge can make you a Topper⚡.mp4"
Deleting original file This 2 Minute Challenge can make you a Topper⚡.f397.mp4 (pass -k to keep)
Deleting original file This 2 Minute Challenge can make you a Topper⚡.f251.webm (pass -k to keep)
Successfully downloaded!


In [ ]:
import base64
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

  



In [28]:
import os
from moviepy.editor import VideoFileClip

from langchain.vectorstores import FAISS

from langchain.schema import HumanMessage
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


# Function to chunk video
def chunk_video(video_path, chunk_length=30):
    """
    Splits video into chunks of `chunk_length` seconds.
    Returns list of video clip objects.
    """
    clip = VideoFileClip(video_path)
    duration = int(clip.duration)
    chunks = []
    for start in range(0, duration, chunk_length):
        end = min(start + chunk_length, duration)
        chunk = clip.subclip(start, end)
        chunks.append(chunk)
    return chunks

# Function to summarize a video chunk
def summarize_chunk(chunk, chunk_index):
    """
    Converts video chunk to text summary using LLM.
    """
    # Export chunk as temporary file
    temp_path = f"temp_chunk_{chunk_index}.mp4"
    chunk.write_videofile(temp_path, codec="libx264", audio_codec="aac", verbose=False, logger=None)

    # Encode chunk to base64
    import base64
    with open(temp_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")

    # Create LLM message
    message = HumanMessage(
        content=[
            {"type": "text", "text": "Summarize the content of this video segment."},
            {"type": "media", "data": encoded, "mime_type": "video/mp4"},
        ]
    )

    response = llm.invoke([message])

    # Cleanup temp file
    os.remove(temp_path)
    return response.content  #  ChatGoogleGenerativeAI style response

# Function to process full video and build vector store
def ingest_video(video_path, chunk_length=30):
    chunks = chunk_video(video_path, chunk_length)
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_chunk(chunk, i)
        summaries.append(summary)
    print("All chunks summarized!")

    # Create vector store
    # Define the model parameters
    model_name = "BAAI/bge-small-en"
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {"normalize_embeddings": True}

    hf_em_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    vector_store = FAISS.from_texts(summaries, hf_em_model)
    return vector_store




In [35]:
video_file = "/content/This 2 Minute Challenge can make you a Topper⚡.mp4"
vector_store = ingest_video(video_file, chunk_length=30)

Summarizing chunk 1/4...
Summarizing chunk 2/4...
Summarizing chunk 3/4...
Summarizing chunk 4/4...


  hf_em_model = HuggingFaceBgeEmbeddings(



All chunks summarized!


Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
template="""
you are a helpfull assistance.
generate answer to user query based on the context provided
here is the user query:{query}
here is the context :{context}
"""
prompt=ChatPromptTemplate.from_template(template)
rag_chain=prompt|llm|StrOutputParser()

In [38]:
# Query the video
query = "What happens in the last part of the video?"
docs = vector_store.similarity_search(query)
response=rag_chain.invoke({"query":query,"context":docs})


In [39]:
response

'In the last part of the video, the speaker encourages viewers to accept a challenge to study for one hour right after they wake up for the next six days. He ends the video by saying, "I want you to win, and I believe in you."'

In [40]:
query = "what heppens in the middle part of the video"
docs = vector_store.similarity_search(query)
response=rag_chain.invoke({"query":query,"context":docs})
response

'In the middle part of the video, the speaker encourages viewers to accept a challenge to study for two minutes every day for six days. The challenge is to plan out what you are going to study early in the morning right after you wake up. The subject you choose should not be your favorite. After six days, try to analyze which of the subjects you were able to study the best in the morning.'

In [42]:
query = "does he mention his youtube channel name if yes what is it?"
docs = vector_store.similarity_search(query)
response=rag_chain.invoke({"query":query,"context":docs})
response

"I'm sorry, but based on the provided text, the speaker's YouTube channel name is not mentioned."

In [43]:
query = "which book was mentioned in this video?"
docs = vector_store.similarity_search(query)
response=rag_chain.invoke({"query":query,"context":docs})
response

'The book mentioned in the video is called "When."'